In [1]:
#from cassandra.cluster import Cluster
import pandas as pd 
import collections
import numpy as np

In [2]:
import csv

col_list = ["movieId", "title","genres"]

df = pd.read_csv("/home/spetz/Downloads/archive/movie.csv", usecols=col_list)
#file.readlines()
for_query2=df
df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [3]:
df=df.drop(columns=["genres"])


In [4]:
columns_list = ["userId", "movieId","rating","timestamp"]
dx=pd.read_csv("/home/spetz/Downloads/archive/rating.csv", usecols=columns_list)
dx=dx.sort_values("movieId")
dx=dx[:10000000]
dx

,userId,movieId,rating,timestamp
14162882,97809,1,3.0,2008-06-11 04:47:11
15347420,106140,1,5.0,2013-01-29 03:33:49
15347355,106138,1,3.0,2002-07-31 15:48:53
10170195,70354,1,4.5,2011-02-13 18:55:40
10170255,70355,1,3.5,2008-01-26 16:56:54
...,...,...,...,...
7816716,53914,2167,4.0,2003-04-09 00:25:00
8114556,55895,2167,4.0,2005-01-30 21:04:59
10750246,74315,2167,2.0,2000-04-05 02:33:11
19621413,135822,2167,1.0,2000-08-02 02:31:03


In [5]:
dx=dx.drop(columns=["userId"])
dx


,movieId,rating,timestamp
14162882,1,3.0,2008-06-11 04:47:11
15347420,1,5.0,2013-01-29 03:33:49
15347355,1,3.0,2002-07-31 15:48:53
10170195,1,4.5,2011-02-13 18:55:40
10170255,1,3.5,2008-01-26 16:56:54
...,...,...,...
7816716,2167,4.0,2003-04-09 00:25:00
8114556,2167,4.0,2005-01-30 21:04:59
10750246,2167,2.0,2000-04-05 02:33:11
19621413,2167,1.0,2000-08-02 02:31:03


In [6]:

dt = dx.sort_values('timestamp').drop_duplicates(['movieId','rating'], keep='last')
dt

,movieId,rating,timestamp
13768535,109,5.0,1996-03-19 04:44:10
19549902,625,5.0,1996-03-28 08:55:52
10594753,752,5.0,1996-05-17 19:08:16
10594712,655,5.0,1996-05-17 19:09:07
12054970,666,1.0,1996-05-22 10:44:15
...,...,...,...
8378393,924,4.5,2015-03-31 05:43:51
8378412,1653,3.5,2015-03-31 05:44:40
8378405,1240,5.0,2015-03-31 05:50:01
8378411,1590,3.5,2015-03-31 05:51:49


In [54]:
new_list = list()
dicts={}
dxx = dx[(dx['timestamp'] >= '2015-01-01') & (dx['timestamp'] <= '2015-03-31')]
for i in range(len(dxx)):
    ar = dxx.loc[(dx['movieId'] == i ) & (dx['rating'] <= 5)]
    rating=ar.apply("rating").sum()
    movies_num=ar['movieId'].value_counts()
    x=sum(rating/movies_num)
    new_list.append(x)
    
    first_value = dxx['movieId'].iat[i]
    dicts[i] = new_list[i]




In [55]:
dicts={x:y for x,y in dicts.items() if y!=0}
#ERWTIMA3
def compare_dict_values(dictionary, given_float):
    return {i: dictionary[i] for i in dictionary if dictionary[i] > given_float}


final_table=compare_dict_values(dicts,0.1)
final_table = pd.DataFrame(list(final_table.items()), columns=['movieId', 'rating'])

final_table

,movieId,rating
0,1,3.860412
1,2,3.128834
2,3,2.854167
3,4,2.000000
4,5,2.824324
...,...,...
1554,2163,1.666667
1555,2164,2.500000
1556,2165,1.750000
1557,2166,3.625000


In [64]:
#QUERY1
df
first_query = df.merge(final_table, on="movieId", how = 'inner')

#ADD TIMESTAMP
dt=dxx.drop_duplicates(subset='movieId', keep="last")
f=dt.drop(['rating'], axis=1)
f

#MERGE RATING WITH TIMESTAMP
timequery = first_query.merge(f, on="movieId", how = 'inner')
#final query1 30-largest values 
timequery = timequery[(timequery['timestamp'] >= '2015-01-01') &( timequery['timestamp'] <= '2015-01-15') ]
timequery=timequery.nlargest(30, ['rating'])
timequery


,movieId,title,rating,timestamp
296,363,"Wonderful, Horrible Life of Leni Riefenstahl, ...",5.000000,2015-01-12 03:48:52
610,823,"Collector, The (La collectionneuse) (1967)",5.000000,2015-01-10 21:13:12
734,1014,Pollyanna (1960),5.000000,2015-01-12 16:18:37
810,1117,"Eighth Day, The (Huitième jour, Le) (1996)",4.750000,2015-01-03 22:32:41
1109,1534,"Bonheur, Le (1965)",4.750000,2015-01-11 15:02:59
631,875,Nothing to Lose (1994),4.500000,2015-01-07 17:55:25
1046,1423,Hearts and Minds (1996),4.500000,2015-01-04 17:46:47
1330,1900,"Children of Heaven, The (Bacheha-Ye Aseman) (1...",4.423077,2015-01-07 02:47:57
33,35,Carrington (1995),4.250000,2015-01-08 04:21:06
64,72,Kicking and Screaming (1995),4.250000,2015-01-05 13:48:04


In [65]:
Query1 = pd.DataFrame(list(dicts.items()), columns=['movieId', 'rating'])

#final_query = Query1.merge(df, on="movieId", how = 'inner')
#final_query = final_query.dropna()

#final_query = final_query.merge(f, on="movieId", how = 'inner')
#final_query = final_query.sort_values('timestamp').drop_duplicates(['movieId','rating','title'], keep='last')


#final_query
Query1

,movieId,rating
0,1,3.860412
1,2,3.128834
2,3,2.854167
3,4,2.000000
4,5,2.824324
...,...,...
1554,2163,1.666667
1555,2164,2.500000
1556,2165,1.750000
1557,2166,3.625000


In [66]:
#QUERYY2
for_query2


final_query2 = for_query2.merge(Query1, on="movieId", how = 'inner')

final_query2


,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.128834
2,3,Grumpier Old Men (1995),Comedy|Romance,2.854167
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.000000
4,5,Father of the Bride Part II (1995),Comedy,2.824324
...,...,...,...,...
1554,2163,Attack of the Killer Tomatoes! (1978),Comedy|Horror,1.666667
1555,2164,Surf Nazis Must Die (1987),Action|Comedy|Drama|Horror,2.500000
1556,2165,Your Friends and Neighbors (1998),Comedy|Drama,1.750000
1557,2166,Return to Paradise (1998),Crime|Drama|Romance|Thriller,3.625000


In [67]:

col_list=['userId','movieId','tag','timestamp']
dyy = pd.read_csv("/home/spetz/Downloads/archive/tag.csv", usecols=col_list)

In [68]:
dyy=dyy.drop(columns=['userId','timestamp'])
dyy
final_query22 = final_query2.merge(dyy, on="movieId", how = 'inner')

In [69]:
final_query22



,movieId,title,genres,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Watched
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,computer animation
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Disney animated feature
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Pixar animation
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,TÃ©a Leoni does not star in this movie
...,...,...,...,...,...
101812,2167,Blade (1998),Action|Horror|Thriller,3.069892,Netflix Finland
101813,2167,Blade (1998),Action|Horror|Thriller,3.069892,action
101814,2167,Blade (1998),Action|Horror|Thriller,3.069892,Bibliothek
101815,2167,Blade (1998),Action|Horror|Thriller,3.069892,vampire


In [70]:
#QUERY3
final_query3=final_query2.drop(columns=["movieId",'genres','rating'])


In [71]:
final_query3


,title
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
...,...
1554,Attack of the Killer Tomatoes! (1978)
1555,Surf Nazis Must Die (1987)
1556,Your Friends and Neighbors (1998)
1557,Return to Paradise (1998)


In [72]:
only_numbers=final_query3['title'].str.extract('(\d+)')
only_numbers.columns=['years']

In [73]:
only_numbers
final_query2['years']= only_numbers['years']
final_query2
query3=final_query2
query3


,movieId,title,genres,rating,years
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.128834,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,2.854167,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.000000,1995
4,5,Father of the Bride Part II (1995),Comedy,2.824324,1995
...,...,...,...,...,...
1554,2163,Attack of the Killer Tomatoes! (1978),Comedy|Horror,1.666667,1978
1555,2164,Surf Nazis Must Die (1987),Action|Comedy|Drama|Horror,2.500000,1987
1556,2165,Your Friends and Neighbors (1998),Comedy|Drama,1.750000,1998
1557,2166,Return to Paradise (1998),Crime|Drama|Romance|Thriller,3.625000,1998


In [74]:
#query4
query4=final_query22
query5=final_query22
query4

,movieId,title,genres,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Watched
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,computer animation
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Disney animated feature
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,Pixar animation
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.860412,TÃ©a Leoni does not star in this movie
...,...,...,...,...,...
101812,2167,Blade (1998),Action|Horror|Thriller,3.069892,Netflix Finland
101813,2167,Blade (1998),Action|Horror|Thriller,3.069892,action
101814,2167,Blade (1998),Action|Horror|Thriller,3.069892,Bibliothek
101815,2167,Blade (1998),Action|Horror|Thriller,3.069892,vampire


In [75]:

timequery.to_csv("/home/spetz/Downloads/archive/query1.csv")
final_query22.to_csv("/home/spetz/Downloads/archive/query2.csv")
query3.to_csv("/home/spetz/Downloads/archive/query3.csv")
query4.to_csv("/home/spetz/Downloads/archive/query4.csv")
query5.to_csv("/home/spetz/Downloads/archive/query5.csv")
#timequery.to_csv("/home/spetz/Downloads/archive/query11.csv")

In [ ]:
from cassandra.cluster import Cluster
cluster =Cluster(['0.0.0.0'], port=9042)
session = cluster.connect("bigdata")